In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
sys.path.append(os.path.abspath(os.path.join(".", "src")))

from audio_processing import audio_import

In [ ]:
loader = audio_import.AudioLoader()

print(loader.directory)
#ensure files can be viewed
os.listdir(loader.directory)

In [ ]:
data = loader.load_audio_file('ex1_distorted.wav')
print(data)

In [ ]:
from audio_processing import distortion_detector_hazel as ddh

clipping_regions = ddh.detect_clipping(data['data'], data['samplerate'], clip_threshold=0.98, plot=False)
print(f"Found {len(clipping_regions)} clipping regions:")

In [ ]:
from clipdetect import detect_clipping
detections, _ = detect_clipping(data['data'])

# Convert sample indices to timestamps using sample rate
sr = data['samplerate']

print(detections)

timestamps = [(detection['start'] / sr, detection['end'] / sr) for detection in detections]

print(f"Found {len(detections)} clipping events:")
for i, (start_time, end_time) in enumerate(timestamps, 1):
    print(f"  {i}. {start_time:.3f}s - {end_time:.3f}s")

In [ ]:
# Prepare data for visualization
audio_data = data['data']
sr = data['samplerate']
time = np.arange(len(audio_data)) / sr

In [ ]:
# Side-by-side comparison of both clipping detection methods
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Calculate average amplitude
avg_amplitude = np.mean(np.abs(audio_data))

# Plot 1: Custom detection method (distortion_detector_hazel)
ax1.plot(time, audio_data, linewidth=0.5, alpha=0.7, color='blue', label='Waveform')
ax1.axhline(avg_amplitude, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='Avg Amplitude')
ax1.axhline(-avg_amplitude, color='green', linestyle='--', linewidth=1.5, alpha=0.7)
for region_time in clipping_regions:
    ax1.axvline(region_time, color='red', alpha=0.7, linewidth=0.8, label='Clipping' if region_time == clipping_regions[0] else '')
ax1.set_ylabel('Amplitude')
ax1.set_title('Clipping Detection - Using original method')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: clipdetect library results
ax2.plot(time, audio_data, linewidth=0.5, alpha=0.7, color='blue', label='Waveform')
ax2.axhline(avg_amplitude, color='green', linestyle='--', linewidth=1.5, alpha=0.7, label='Avg Amplitude')
ax2.axhline(-avg_amplitude, color='green', linestyle='--', linewidth=1.5, alpha=0.7)
for detection in detections:
    start_time = detection['start'] / sr
    end_time = detection['end'] / sr
    ax2.axvspan(start_time, end_time, color='red', alpha=0.5, label='Clipping' if detection == detections[0] else '')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Amplitude')
ax2.set_title('Clipping Detection - Using clipdetect library')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nComparison Summary:")
print(f"  Average amplitude: {avg_amplitude:.4f}")
print(f"  Custom method: {len(clipping_regions)} sample points detected")
print(f"  clipdetect library: {len(detections)} regions detected")